In [52]:
import numpy as np
import pandas as pd
from transaction_dates import *
from imiona import *
from collections import Counter
from samochody import *
from czesci import *

# Pracownicy

In [53]:
birth_dates_lims = [1960, 1995]
begining_date = "2014-01-01"
end_date = "2018-01-01"

def rind(a, b):
    return np.random.randint(a, b)

jobs_list = np.array(["menadżer", "sprzedawca", "mechanik", "pomocnik mechanika", 
                       "diagnosta samochodowy", "sprzątający", "informatyk", "księgowy"])
jobs_amount = np.array([1, rind(2, 5), rind(3, 6), rind(1, 3),
                        rind(1, 3), rind(1, 3), rind(1, 3), rind(0, 2)])
fixed_jobs = np.array([0, 1, 2, np.sum(jobs_amount[:2]), np.sum(jobs_amount[0:6])]) # zaczynamy z szefem, dwoma sprzedawcami i mechanikiem

employment_dates = random_transaction_dates(begining_date, end_date, np.sum(jobs_amount))
employment_dates[fixed_jobs] = begining_date  

wages_lims = [(60, 75), (45, 60), (45, 60), (35, 45),
              (47, 60), (35, 40), (40, 60), (40, 53)]

employees_df = pd.DataFrame(columns=["id", "imię", "nazwisko", "pesel",
                                     "miasto", "adres", "telefon", "email", "data_zatrudnienia",
                                     "data_zwolnienia", "stanowisko", "płaca[zł/h]"])
for (i, job) in enumerate(jobs_list):
    c = 0
    for j in range(jobs_amount[i]):
        employment_date = employment_dates[np.sum(jobs_amount[:i]) + j]
        if np.random.rand() <= 0.5:
            first_name, last_name = random_name_man()
            pesel, birth_date = random_pesel_date_man(birth_dates_lims)
            sex = "M"
            city = random_city(prob=0.7)
            address = random_street()
            phone = random_phone()
            #email = random_email(first_name, last_name, birth_date[-4:])
            email = employee_email(first_name, last_name)
            employment_date = employment_dates[np.sum(jobs_amount[:i]) + j]
            wage = np.random.choice(np.arange(wages_lims[i][0], wages_lims[i][1], 0.1))
        else:
            first_name, last_name = random_name_woman()
            pesel, birth_date = random_pesel_date_woman(birth_dates_lims)
            sex = "K"
            city = random_city(prob=0.7)
            address = random_street()
            phone = random_phone()
            #email = random_email(first_name, last_name, birth_date[-4:])
            email = employee_email(first_name, last_name)
            employment_date = employment_dates[np.sum(jobs_amount[:i]) + j]
            wage = np.random.choice(np.arange(wages_lims[i][0], wages_lims[i][1], 0.1))
        if np.random.rand() <= (jobs_amount[i]-1)/10 * 1/(1+c):
            dismissal_date = random_transaction_dates(str(employment_date), end_date, 1)[0]
            c += 1
        else:
            dismissal_date = None

        employees_df.loc[np.sum(jobs_amount[:i]) + j] = [0, first_name, last_name, pesel,
                                                         city, address, phone, email, employment_date,
                                                         dismissal_date, job, wage]

employees_df = employees_df.sort_values(by="data_zatrudnienia")   
employees_df["id"] =  np.arange(1, len(employees_df)+1)

In [54]:
employees_df

,id,imię,nazwisko,pesel,miasto,adres,telefon,email,data_zatrudnienia,data_zwolnienia,stanowisko,płaca[zł/h]
0,1,Ari,Wróbel,63092489294,Gołdap,Aleja Lipowa 54/14,48203288081,ari.wrobel@kuba.pl,2014-01-01,None,menadżer,72.8
1,2,Andrzej,Jędryka,93051843038,Jaworzno,ul. Strażacka 63,48354575816,andrzej.jedryka@kuba.pl,2014-01-01,None,sprzedawca,54.9
2,3,Anastazja,Kuta,75092589140,Wrocław,ul. Leśny Zaułek 32/49,48899405167,anastazja.kuta@kuba.pl,2014-01-01,None,sprzedawca,57.0
5,4,Sebastian,Pawlicki,65081993771,Wrocław,ul. Elizy Orzeszkowej 19,48427486576,sebastian.pawlicki@kuba.pl,2014-01-01,None,mechanik,57.8
13,5,Nela,Ślimak,68081992160,Wrocław,ul. Słoneczna 92,48334841424,nela.slimak@kuba.pl,2014-01-01,None,informatyk,56.6
3,6,Aneta,Stanovych,82040656003,Wrocław,ul. M. Kopernika 11/33,48961670250,aneta.stanovych@kuba.pl,2015-06-28,2017-10-07,sprzedawca,47.6
4,7,Bożena,Jastrzębska,72111677062,Wrocław,ul. Grodziska 31/24,48678712116,bozena.jastrzebska@kuba.pl,2015-07-10,None,sprzedawca,59.5
6,8,Michał,Tatarczak,69033111855,Kruszwica,ul. Stanisława Augusta Poniatowskiego 78,48997497407,michal.tatarczak@kuba.pl,2016-04-30,None,mechanik,49.3
7,9,Nikola,Blyzniak,84032952346,Wrocław,ul. Strzelecka 91,48034374336,nikola.blyzniak@kuba.pl,2016-06-09,None,mechanik,52.6
8,10,Lena,Bojarzyńska,75022059163,Wrocław,ul. Słoneczna 16/6,48508142280,lena.bojarzynska@kuba.pl,2016-06-23,None,pomocnik mechanika,35.3


# Transakcje

In [55]:
no_transactions = 2000
begining_buy = 20

dates = random_transaction_dates("2014-01-01", "2020-12-31", no_transactions)

In [56]:
client_ids = np.zeros(len(dates), dtype=int) # array with client ids for table with transactions

j = 1
for i in range(len(dates)):
    prob = i/(2*len(dates))
    if np.random.rand() <= prob:
        client_ids[i] = np.random.choice(np.unique(client_ids[:i]))
    else:
        client_ids[i] = j
        j += 1

In [57]:
car_ids = np.zeros(len(dates), dtype=int) # array with car ids for table with transactions
car_ids[:begining_buy] = np.arange(1, 21)

client_ids = np.zeros(len(dates), dtype=int) # array with client ids for table with transactions
client_ids[:begining_buy] = np.arange(1, 21)

transactions_types = np.zeros(len(dates))

clients_cars = {}
our_cars = np.arange(1, 21).tolist()

jp = 21
jc = 21
for i in range(20, len(dates)):
    weights = np.array([1.5*len(clients_cars)/max(1, len(our_cars)) , len(our_cars)*50, len(clients_cars)/(5*max(1, len(our_cars)))])
    weights = weights/np.sum(weights)
    transaction_type = np.random.choice(np.arange(3), p=weights) # 0-buy, 1-sell, 2-service
    transactions_types[i] = transaction_type
    if transaction_type == 0:
        prob = i/(2*len(dates)) 
        if np.random.rand() <= prob: #client that already is in database
            client_ids[i] = np.random.choice(np.unique(client_ids[:i]))
            if client_ids[i] in clients_cars and np.random.rand() <= 0.6 and len(clients_cars[client_ids[i]]) >= 1:
                car_ids[i] = np.random.choice(clients_cars[client_ids[i]])
                clients_cars[client_ids[i]].remove(car_ids[i])
            else:
                car_ids[i] = jc
                jc += 1
            our_cars.append(car_ids[i])
        else:
            client_ids[i] = jp
            car_ids[i] = jc
            our_cars.append(car_ids[i])
            jp += 1
            jc += 1

    if transaction_type == 1:
        prob = i/(2*len(dates))
        if np.random.rand() <= prob: #client that already is in database
            client_ids[i] = np.random.choice(np.unique(client_ids[:i]))
            car_ids[i] = np.random.choice(our_cars)
            if client_ids[i] in clients_cars:
                clients_cars[client_ids[i]].append(car_ids[i])
            else:
                clients_cars[client_ids[i]] = [car_ids[i]]
            our_cars.remove(car_ids[i])
        else:
            client_ids[i] = jp
            car_ids[i] = np.random.choice(our_cars)
            clients_cars[client_ids[i]] = [car_ids[i]]
            our_cars.remove(car_ids[i])
            jp += 1
    
    if transaction_type == 2:
        prob = i/(2*len(dates))
        if np.random.rand() <= prob: #client that already is in database
            client_ids[i] = np.random.choice(np.unique(client_ids[:i]))
            if client_ids[i] in clients_cars:
                if np.random.rand() <= 0.6 and len(clients_cars[client_ids[i]]) >= 1:
                    car_ids[i] = np.random.choice(clients_cars[client_ids[i]])
                else:
                    car_ids[i] = jc
                    clients_cars[client_ids[i]].append(car_ids[i])
                    jc += 1
            else:
                car_ids[i] = jc
                clients_cars[client_ids[i]] = [car_ids[i]]
                jc += 1
        else:
            client_ids[i] = jp
            car_ids[i] = jc
            clients_cars[client_ids[i]] = [car_ids[i]]
            jp += 1
            jc += 1

In [58]:
buy_indexes = np.where(transactions_types == 0)
sell_indexes = np.where(transactions_types == 1)
service_indexes = np.where(transactions_types == 2)

In [59]:
transaction_dates_buy = dates[buy_indexes]
transaction_dates_sell = dates[sell_indexes]
transaction_dates_service = dates[service_indexes]

client_ids_buy = client_ids[buy_indexes]
client_ids_sell = client_ids[sell_indexes]
client_ids_service = client_ids[service_indexes]

car_ids_buy = car_ids[buy_indexes]
car_ids_sell = car_ids[sell_indexes]
car_ids_service = car_ids[service_indexes]

In [60]:
buy_employee_id = np.zeros(len(transaction_dates_buy))

for i in range(len(transaction_dates_buy)):
    filter1 = employees_df["stanowisko"] == "sprzedawca"
    filter2 = employees_df["data_zatrudnienia"] <= transaction_dates_buy[i]
    filter3 = pd.notna(employees_df["data_zwolnienia"])
    filter4 = employees_df["data_zwolnienia"] < transaction_dates_buy[i]
    buy_employee_id[i] = np.random.choice(employees_df['id'][filter1 & filter2 & ~(filter3 & filter4)])

In [61]:
sell_employee_id = np.zeros(len(transaction_dates_sell))

for i in range(len(transaction_dates_sell)):
    filter1 = employees_df["stanowisko"] == "sprzedawca"
    filter2 = employees_df["data_zatrudnienia"] <= transaction_dates_sell[i]
    filter3 = pd.notna(employees_df["data_zwolnienia"])
    filter4 = employees_df["data_zwolnienia"] < transaction_dates_sell[i]
    sell_employee_id[i] = np.random.choice(employees_df['id'][filter1 & filter2 & ~(filter3 & filter4)])

In [62]:
service_employee_id = np.zeros(len(transaction_dates_service))

for i in range(len(transaction_dates_service)):
    filter1 = employees_df["stanowisko"] == "mechanik"
    filter2 = employees_df["data_zatrudnienia"] <= transaction_dates_service[i]
    filter3 = pd.notna(employees_df["data_zwolnienia"])
    filter4 = employees_df["data_zwolnienia"] < transaction_dates_service[i]
    service_employee_id[i] = np.random.choice(employees_df['id'][filter1 & filter2 & ~(filter3 & filter4)])


# klienci

In [63]:
year_lims = [1970, 2010]
transaction_counter = Counter(client_ids)

clients_df = pd.DataFrame(columns=["id", "imię", "nazwisko", "płeć", "PESEL", "miasto", "adres", "karta_lojalnościowa", "telefon", "email"])
unique_clients = np.unique(client_ids)
for i in range(len(unique_clients)):
    if np.random.rand() <= 0.5: # chłop probability
        id_ = unique_clients[i]
        first_name, last_name = random_name_man()
        if id_ in client_ids_buy or id_ in client_ids_sell:
            pesel, birth_date = random_pesel_date_man(year_lims)
            year = birth_date[-4:]
        else:
            pesel = None
            birth_date = None
            year = str(np.random.randint(year_lims[0], year_lims[1]))
        sex = "M"
        city = random_city()
        address = random_street()
        if transaction_counter[id_] >= 3:
            loyalty_card = True
        else:
            loyalty_card = False
        phone = random_phone()
        email = random_email(first_name, last_name, year)
        clients_df.loc[i] = [unique_clients[i], first_name, last_name, sex, pesel, city, address, loyalty_card, phone, email]
    else:
        id_ = unique_clients[i]
        first_name, last_name = random_name_woman()
        if id_ in client_ids_buy or id_ in client_ids_sell:
            pesel, birth_date = random_pesel_date_man(year_lims)
            year = birth_date[-4:]
        else:
            pesel = None
            birth_date = None
            year = str(np.random.randint(year_lims[0], year_lims[1]))
        sex = "F"
        city = random_city()
        address = random_street()
        if transaction_counter[id_] >= 3:
            loyalty_card = True
        else:
            loyalty_card = False
        phone = random_phone()
        email = random_email(first_name, last_name, year)
        clients_df.loc[i] = [unique_clients[i], first_name, last_name, sex, pesel, city, address, loyalty_card, phone, email]

In [64]:
clients_df

,id,imię,nazwisko,płeć,PESEL,miasto,adres,karta_lojalnościowa,telefon,email
0,1,Dominik,Tiachnybok,M,71123138938,Mrocza,ul. Marii Rodziewiczówny 73/14,False,48478734292,tiachnydominik8500@gmail.com
1,2,Maciej,Strzykała,M,94080698233,Wrocław,ul. Radosna 35/51,False,48640955540,u8drt0b0u284h0k@gmail.com
2,3,Edyta,Gajda,F,98082290417,Kędzierzyn-Koźle,ul. Spacerowa 75,False,48620864962,u0z54p48z222y0dro@wp.pl
3,4,Grażyna,Plichta,F,02231270830,Tuchola,ul. Michałowskiego 54,False,48756879602,g.plichta6174@o2.pl
4,5,Barbara,Kinalska,F,79082746450,Łódź,ul. Spokojna 23,False,48541389059,kinalska795@gmail.com
...,...,...,...,...,...,...,...,...,...,...
1510,1511,Helena,Łukawska,F,10250656096,Wrocław,ul. Projektowana 50/50,False,48982182874,lukawska20106@wp.pl
1511,1512,Krzysztof,Soja,M,91070957871,Wrocław,ul. Tęczowa 25/35,False,48636139444,gi2qvh688gur@gazeta.pl
1512,1513,Marta,Laskowska,F,01310146910,Gdynia,ul. Dębowa 70/14,False,48342026193,laskowska0@gmail.com
1513,1514,Cezary,Lewecki,M,88041007671,Leśnica,ul. Słoneczna 7/22,False,48422743237,clewecki1988@gmail.com


In [65]:
np.sum(clients_df['karta_lojalnościowa'])

np.int64(78)

In [66]:
np.sum(pd.isna(clients_df["PESEL"]))

np.int64(60)

# auta

In [67]:
rng = np.random.default_rng()

data_car, data_bike = get_car_data()
make_list_car, model_list_car, car_weights = get_car_weights()
make_list_bike, model_list_bike, bike_weights = get_bike_weights()

In [68]:
def create_car(i):

    is_car = rng.random() < 0.9
    colors = np.array(('czarny', 'biały', 'szary', 'srebrny', 'niebieski', 'czerwony', 'brązowy', 'złoty', 'żółty', 'inny'))
    colors_weights = np.array((0.22, 0.19, 0.18, 0.15, 0.10, 0.09, 0.03, 0.02, 0.01, 0.01))

    if is_car:
        index = rng.choice(len(make_list_car), p=car_weights)
        make, model = make_list_car[index], model_list_car[index]
        weights = data_car[(data_car['make'] == make) & (data_car['model'] == model)]
        body = np.array(('SUV', 'Hatchback', 'Sedan', 'Kombi', 'Coupe', 'Van', 'Minivan', 'Kabriolet'))
        body_weights = np.array((0.25, 0.25, 0.23, 0.17, 0.06, 0.03, 0.009, 0.001))
    else:
        index = rng.choice(len(make_list_bike), p=bike_weights)
        make, model = make_list_bike[index], model_list_bike[index]
        weights = False
        body = False
        body_weights = False

    wheels = 4 if is_car else 2
    year = get_year()
    gear = get_gear(weights, is_car)
    fuel = get_fuel(weights, is_car)
    post_accident = get_post_accident()
    hp = get_hp(weights, is_car)
    color = get_color(colors, colors_weights)
    body_type = get_body_type(body, body_weights, is_car)
    engine_capacity = get_engine_capacity(hp, is_car)
    doors = get_doors(is_car)
    seats = get_num_of_seats(is_car)
    available = i+1 in our_cars
    
    return [i+1, make, model, wheels, year, body_type, post_accident, color, fuel, engine_capacity, hp, gear, doors, seats, available]


def create_car_table(n):
    
    df = pd.DataFrame(columns=['id', 'marka', 'model', 'liczba_kół', 'rok_produkcji', 'typ_nadwozia', 'czy_powypadkowy', 'kolor', 'typ_paliwa', 'pojemność_silnika', 'moc', 'skrzynia_biegów', 'liczba_drzwi', 'liczba_miejsc', 'czy_dostępny'])
    for i in range(n):
        row = create_car(i)
        df.loc[i] = row
    return df


In [69]:
df_samochody = create_car_table(max(car_ids))

In [70]:
df_samochody

,id,marka,model,liczba_kół,rok_produkcji,typ_nadwozia,czy_powypadkowy,kolor,typ_paliwa,pojemność_silnika,moc,skrzynia_biegów,liczba_drzwi,liczba_miejsc,czy_dostępny
0,1,Ford,Fiesta,4,2001,Kombi,False,czarny,Diesel,1.3,80,Manualna,5,5,False
1,2,Mercedes-Benz,B 180,4,2012,Kombi,False,brązowy,Benzyna,2.0,122,Manualna,3,5,False
2,3,Citroen,Berlingo,4,2009,SUV,True,inny,Benzyna,1.5,99,Manualna,5,5,False
3,4,Audi,Q5,4,2009,Sedan,False,czarny,Diesel,3.1,190,Automatyczna,5,5,False
4,5,Citroen,C3,4,2010,Sedan,False,srebrny,Benzyna,1.1,68,Manualna,5,5,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
971,972,Audi,Q5,4,2001,Kombi,False,złoty,Diesel,2.8,190,Automatyczna,5,5,False
972,973,Volkswagen,Golf,4,2010,Hatchback,False,czarny,Diesel,3.0,184,Manualna,5,5,False
973,974,Toyota,Prius,4,2005,Sedan,False,biały,Elektryczny/Benzyna,1.5,99,Automatyczna,5,5,True
974,975,Renault,Clio,4,2005,Hatchback,False,czarny,Diesel,1.5,101,Manualna,3,5,True


# Usługi

In [71]:
service_employee = rng.choice(employees_df[employees_df['stanowisko'] == 'mechanik']['id'], len(car_ids_service))
service_ids = np.arange(len(transaction_dates_service)) + 1

df_services = pd.DataFrame()
df_services['id'] = service_ids
df_services['id_samochodu'] = car_ids_service
df_services['id_klienta'] = client_ids_service
df_services['id_pracownika'] = service_employee
df_services['data_zaksiegowania'] = transaction_dates_service

# Zakup samochodów

In [72]:
def car_purchase_price(hp, yr_bought, yr_prod, post_acc):
    return np.round(hp * 1000 / np.sqrt(int(yr_bought) - int(yr_prod)) * (0.75 if post_acc else 1) * rng.uniform(0.8, 1.2), -2)

In [73]:
buy_employee = rng.choice(employees_df[employees_df['stanowisko'] == 'sprzedawca']['id'], len(car_ids_buy))
buy_ids = np.arange(len(transaction_dates_buy)) + 1

df_buy = pd.DataFrame()
df_buy['id'] = buy_ids
df_buy['id_samochodu'] = car_ids_buy
df_buy['id_klienta'] = client_ids_buy
df_buy['id_pracownika'] = buy_employee
df_buy['data_zaksiegowania'] = transaction_dates_buy

In [74]:

service_date_fix = pd.merge(df_samochody[['id', 'rok_produkcji']], df_services[['id_samochodu', 'data_zaksiegowania']], how='right', left_on='id', right_on='id_samochodu')
service_date_fix['data_zaksiegowania'] = service_date_fix['data_zaksiegowania'].str[:4]

buy_date_fix = pd.merge(df_samochody[['id', 'rok_produkcji']], df_buy[['id_samochodu', 'data_zaksiegowania']], how='right', left_on='id', right_on='id_samochodu')
buy_date_fix['data_zaksiegowania'] = buy_date_fix['data_zaksiegowania'].str[:4]

for i in range(len(buy_date_fix)):
    if str(buy_date_fix['rok_produkcji'].loc()[i]) >= str(buy_date_fix['data_zaksiegowania'].loc()[i]):
        car_index = buy_date_fix.loc[i, 'id_samochodu']
        df_samochody.loc[car_index-1, 'rok_produkcji'] = str(int(df_samochody.loc[car_index-1, 'rok_produkcji']) - 10)

for i in range(len(service_date_fix)):
    if str(service_date_fix['rok_produkcji'].loc()[i]) >= str(service_date_fix['data_zaksiegowania'].loc()[i]):
        car_index = service_date_fix.loc[i, 'id_samochodu']
        df_samochody.loc[car_index-1, 'rok_produkcji'] = str(int(df_samochody.loc[car_index-1, 'rok_produkcji']) - 10)

C:\Users\oskar.LAPTOP-KARLNF81\AppData\Local\Temp\ipykernel_16980\3762543032.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2005' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_samochody.loc[car_index-1, 'rok_produkcji'] = str(int(df_samochody.loc[car_index-1, 'rok_produkcji']) - 10)


In [75]:
purchase_prices = []
for j, i in enumerate(car_ids_buy):
    row = df_samochody.loc()[i-1]
    year_bought = transaction_dates_buy[j][:4]
    purchase_prices.append(car_purchase_price(row['moc'], year_bought, row['rok_produkcji'], row['czy_powypadkowy']))

df_buy['cena'] = purchase_prices

C:\Users\oskar.LAPTOP-KARLNF81\AppData\Local\Temp\ipykernel_16980\3354836249.py:2: RuntimeWarning: divide by zero encountered in scalar divide
  return np.round(hp * 1000 / np.sqrt(int(yr_bought) - int(yr_prod)) * (0.75 if post_acc else 1) * rng.uniform(0.8, 1.2), -2)


# Sprzedaż samochodów

In [76]:
sell_employee = rng.choice(employees_df[employees_df['stanowisko'] == 'sprzedawca']['id'], len(car_ids_sell))
sell_ids = np.arange(len(transaction_dates_sell)) + 1

df_sell = pd.DataFrame()
df_sell['id'] = sell_ids
df_sell['id_samochodu'] = car_ids_sell
df_sell['id_klienta'] = client_ids_sell
df_sell['id_pracownika'] = sell_employee
df_sell['data_zaksiegowania'] = transaction_dates_sell

In [77]:
df_sell

,id,id_samochodu,id_klienta,id_pracownika,data_zaksiegowania
0,1,11,21,6,2014-01-24
1,2,8,22,7,2014-01-24
2,3,14,23,2,2014-01-25
3,4,15,24,2,2014-01-26
4,5,7,25,6,2014-01-28
...,...,...,...,...,...
946,947,966,1512,7,2020-12-12
947,948,973,1513,3,2020-12-15
948,949,32,320,2,2020-12-17
949,950,81,1514,2,2020-12-22


In [78]:
sell_prices = []
for i in car_ids_sell:
    b_price = df_buy[df_buy['id_samochodu'] == i]['cena'].values[0]
    print(b_price)
    sell_prices.append(np.round(b_price * rng.uniform(1.2, 1.5), -2))

df_sell['cena'] = sell_prices

26000.0
24700.0
64700.0
28300.0
57700.0
91600.0
32300.0
38000.0
21400.0
57600.0
46400.0
66200.0
47900.0
32700.0
26700.0
32500.0
32200.0
82900.0
53400.0
28700.0
70500.0
57700.0
89800.0
96700.0
inf
63400.0
30400.0
47600.0
30300.0
64300.0
46000.0
218800.0
29000.0
49500.0
44500.0
65000.0
36100.0
inf
43300.0
23300.0
30300.0
36700.0
28800.0
25200.0
130200.0
28100.0
119200.0
34500.0
59800.0
42800.0
49600.0
18400.0
52100.0
88300.0
42500.0
24500.0
22500.0
33100.0
17500.0
40700.0
248400.0
54900.0
55900.0
40900.0
30700.0
27400.0
9100.0
72600.0
50700.0
80000.0
47100.0
19400.0
24900.0
24100.0
28700.0
28500.0
31100.0
19800.0
49400.0
inf
65800.0
28600.0
50800.0
41300.0
18800.0
24400.0
22100.0
73000.0
41400.0
30000.0
96700.0
75300.0
61800.0
34200.0
47200.0
22900.0
inf
22200.0
26200.0
116600.0
32700.0
21900.0
36400.0
31600.0
44200.0
34900.0
55900.0
82800.0
28400.0
50700.0
42400.0
inf
55500.0
83200.0
26800.0
80500.0
39900.0
14400.0
117900.0
9000.0
31300.0
39600.0
40100.0
57000.0
33900.0
inf
101400.0
643

In [79]:
df_sell

,id,id_samochodu,id_klienta,id_pracownika,data_zaksiegowania,cena
0,1,11,21,6,2014-01-24,31700.0
1,2,8,22,7,2014-01-24,33200.0
2,3,14,23,2,2014-01-25,94000.0
3,4,15,24,2,2014-01-26,40800.0
4,5,7,25,6,2014-01-28,70000.0
...,...,...,...,...,...,...
946,947,966,1512,7,2020-12-12,20300.0
947,948,973,1513,3,2020-12-15,88700.0
948,949,32,320,2,2020-12-17,275600.0
949,950,81,1514,2,2020-12-22,24700.0


In [80]:
np.sum(df_sell["cena"] == float('inf')) # wychodzą inf wiec trzeba poprawić

np.int64(17)

# Użyte części

In [81]:
data_parts = get_data_parts()
data_parts_filtered = data_parts[data_parts['motor'] == True]

def get_parts_indexes(car_id):
    if df_samochody.loc[car_id-1].values[3] == 2:
        parts_set = data_parts_filtered[['id','część']]
    else:
        parts_set = data_parts[['id','część']]
    
    parts_indexes = parts_set['id']
    num_parts = rng.choice([1, 2, 3], p=[0.6, 0.3, 0.1])
    indexes = rng.choice(parts_indexes, size=num_parts)
    return indexes

all_car_parts_indexes = [(i, get_parts_indexes(i)) for i in car_ids_service]
print(all_car_parts_indexes)

[(np.int64(41), array([56, 56,  8])), (np.int64(45), array([49,  7])), (np.int64(74), array([0, 5])), (np.int64(89), array([47])), (np.int64(94), array([38])), (np.int64(99), array([ 9, 35])), (np.int64(107), array([2])), (np.int64(115), array([42])), (np.int64(119), array([42])), (np.int64(120), array([18])), (np.int64(126), array([44])), (np.int64(137), array([0])), (np.int64(139), array([8])), (np.int64(164), array([38, 11])), (np.int64(209), array([53, 54, 38])), (np.int64(211), array([ 9, 11])), (np.int64(213), array([51])), (np.int64(214), array([52])), (np.int64(215), array([55])), (np.int64(222), array([50])), (np.int64(224), array([24, 21])), (np.int64(229), array([46])), (np.int64(287), array([52])), (np.int64(292), array([ 4, 28])), (np.int64(299), array([ 6, 59])), (np.int64(306), array([56])), (np.int64(325), array([29])), (np.int64(333), array([36, 59, 54])), (np.int64(349), array([62, 21])), (np.int64(355), array([43])), (np.int64(368), array([63,  5])), (np.int64(371), 

In [83]:
#df_samochody.loc()[1] - samochód o indeksie 0

def create_part(id, service_id, car):
    part = data_parts.loc()[id]
    if type(part['atrybuty']) != float:
        attributes = part['atrybuty'].split()
        for i in range(len(attributes)):
            if attributes[i] == 'marka':
                attributes[i] = car['marka']
            if attributes[i] == 'model':
                attributes[i] = car['model']
            if attributes[i] == 'kolor':
                attributes[i] = car['kolor']
            if attributes[i] == 'rozmiar':
                attributes[i] = 'R' + str(rng.choice([15, 16, 17]))
            if attributes[i] == 'material':
                attributes[i] = rng.choice(['skóra', 'materiał'], p=[0.2, 0.8])
            if attributes[i] == 'pojemnosc':
                attributes[i] = str(car['pojemność_silnika'])
            if attributes[i] == 'skrzynia':
                attributes[i] = str(car['skrzynia_biegów'])
        price = int(np.floor(part['cena']*car['pojemność_silnika']/1.8 * rng.uniform(0.8, 1.2)))
    else:
        attributes = ''
        price = part['cena']
    quantity = part['maxcount']
    return part['część'], ' '.join(attributes), price, service_id, quantity

parts = []
for service_id, element in enumerate(all_car_parts_indexes):
    car_id, parts_id = element
    car = df_samochody.loc()[car_id-1]
    for part_id in parts_id:
        parts.append(create_part(part_id, service_id+1, car))

parts = np.array(parts)
parts_df = pd.DataFrame(parts, columns=['nazwa', 'atrybuty', 'cena', 'id_usługi', 'ilość']).drop_duplicates(['nazwa', 'atrybuty'])
parts_df['id'] = np.arange(1, len(parts_df)+1)
parts_df = parts_df[['id', 'nazwa', 'atrybuty', 'cena', 'id_usługi', 'ilość']]

parts = pd.DataFrame(parts, columns=['nazwa', 'atrybut', 'cena', 'id_usługi', 'ilość'])

In [84]:
parts

,nazwa,atrybut,cena,id_usługi,ilość
0,Kierownica,Renault Clio,73,1,1
1,Kierownica,Renault Clio,56,1,1
2,Wewnętrzna klamka drzwi,Renault Clio,6,1,1
3,Filtr paliwa,Volkswagen T5 Kombi,393,2,1
4,Zewnętrzna klamka drzwi,Volkswagen T5 Kombi,7,2,1
...,...,...,...,...,...
139,Rozrząd,,2500,91,1
140,Maska,SEAT Ibiza szary,525,91,1
141,Silnik,Fiat Panda 1.2,1913,92,1
142,Zespół układu LPG,,500,93,1


# Wyposażenie

In [ ]:
eq_len = len(parts_df)
equipment_id = parts_df['id']
equipment_name = parts_df['nazwa']
equipment_attr = parts_df['atrybuty']
equipment_price = parts_df['cena']

df_equipment = pd.DataFrame()
df_equipment['id'] = equipment_id
df_equipment['nazwa'] = equipment_name
df_equipment['atrybut'] = equipment_attr
df_equipment['cena_części'] = equipment_price.astype(float)

In [ ]:
df_equipment

,id,nazwa,atrybut,cena_części
0,1,Zagłówek,Mazda CX-5,25.0
1,2,Dywan,Audi Q5,57.0
2,3,Akumulator,Audi Q5,1472.0
3,4,Maska,BMW X4 M srebrny,1584.0
4,5,Poduszka powietrzna,,100.0
...,...,...,...,...
156,124,Opona zimowa,R16,192.0
157,125,Pompa paliwowa,Citroen C3,214.0
159,126,Pompa paliwowa,Royal Enfield Electra 5 S,72.0
160,127,Zagłówek,Audi RS3,61.0


# Usługi - uzupełnienie

In [ ]:
service_prices = np.zeros(len(service_ids))
for i in range(len(parts)):
    row = parts.loc()[i]
    service_index = int(row['id_usługi'])
    service_prices[service_index-1] += float(row['cena']) * int(row['ilość'])

df_services['cena'] = np.round(service_prices*rng.uniform(1.1, 1.4), -1).astype(int)

In [ ]:
df_services

,id,id_samochodu,id_klienta,id_pracownika,data_zaksiegowania,cena
0,1,35,69,4,2014-04-23,70
1,2,43,83,7,2014-05-14,2190
2,3,44,85,4,2014-05-15,2320
3,4,45,86,7,2014-05-15,70
4,5,49,92,8,2014-05-22,60
...,...,...,...,...,...,...
110,111,141,260,4,2020-10-27,1410
111,112,953,1399,7,2020-11-07,110
112,113,740,693,8,2020-11-20,170
113,114,964,700,4,2020-12-13,140


# Użyte części - uzupełnienie

In [ ]:
used_parts_df = parts.merge(df_equipment, on=['nazwa', 'atrybut'])
used_parts_df.rename(columns={'id': 'id_części'}, inplace=True)
used_parts_df['ilość'] = pd.to_numeric(used_parts_df['ilość'])
ids = np.arange(len(used_parts_df))+1
used_parts_df['id'] = ids
used_parts_df = used_parts_df[['id', 'id_usługi', 'id_części', 'ilość']]

In [ ]:
used_parts_df

,id,id_usługi,id_części,ilość
0,1,1,1,2
1,2,2,2,2
2,3,2,3,1
3,4,3,4,1
4,5,3,5,1
...,...,...,...,...
158,159,112,8,1
159,160,112,126,1
160,161,113,127,2
161,162,114,24,1


# Zakup części

In [ ]:
def get_service_date(part_id):
    
    index_used_part = used_parts_df[used_parts_df['id_części'] == part_id].index[0]
    service_date = df_services[df_services['id'] == index_used_part]['data_zaksiegowania']
    return service_date
    

buy_parts_quantity = used_parts_df.groupby(['id_części'])['ilość'].sum().reset_index()
parts_purchase_pd = pd.DataFrame(columns=['id', 'id_części', 'koszt', 'data_zakupu', 'ilość', 'data_zaksięgowania'])

for i in range(len(buy_parts_quantity)):
    part_id = buy_parts_quantity['id_części'][i]
    quantity = buy_parts_quantity['ilość'][i]
    price = df_equipment['cena_części'][df_equipment['id'] == part_id].to_numpy()
    if price > 1000:
        quantity += rng.random() < 0.5
    elif price > 500:
        quantity += rng.choice([0, 1, 2], p=[0.2, 0.5, 0.3])
    else:
        quantity += rng.binomial(25, 0.15)
    total_cost = np.floor(quantity * price * rng.uniform(0.75, 0.95))
    service_date = get_service_date(part_id)

    ##### DO DODANIA
    # generowanie daty zakupu części na podstawie purchase_date
    # generowanie daty zaksięgowania transakcji na podstawie purchase_date
    #####
    
    # row = [i+1, part_id, total_cost, purchase_date, quantity, posting_date]
    # parts_purchase_pd.loc[i] = row


In [ ]:
# parts_purchase_pd